In [1]:
import gymnasium as gym
from gymnasium.envs.registration import register
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

In [2]:
register(
    id="TowerClimb-v0",
    entry_point="tower_climb.tower_climb_env:TowerClimbEnv",
)

In [3]:

env = gym.make("TowerClimb-v0")

# Opcjonalnie: sprawdzenie środowiska
check_env(env, warn=True)

model = PPO("MultiInputPolicy", env, verbose=1)
model.learn(total_timesteps=100_000)
print(model.get_parameters())

model.save("ppo_tower_climb")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 1253 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 767         |
|    ep_rew_mean          | 280         |
| time/                   |             |
|    fps                  | 833         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010718762 |
|    clip_fraction        | 0.105       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 0.00592     |
|    learning_rate        | 0.0003      |
|    loss               

In [ ]:
model = PPO.load("ppo_tower_climb")

env = gym.make("TowerClimb-v0", render_mode="human")
obs, _ = env.reset()


done = False
while not done:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    #print(f"Reward: {reward}, Done: {done}, Info: {info}, Action: {action}, Obs: {obs}")
    env.render()

C:\Users\krzys\AppData\Roaming\Python\Python312\site-packages\gymnasium\utils\passive_env_checker.py:334: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
